In [50]:
import pandas as pd
import networkx as nx
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [51]:
edges = pd.read_csv("/kaggle/input/cleaned/edges_cleaned.csv")
nodes = pd.read_csv("/kaggle/input/node-final/nodes.csv")

print(f"→ Số cạnh (edges): {len(edges):,} cạnh")
print(f"→ Số nút (nodes): {len(nodes):,} nút")

→ Số cạnh (edges): 10,257 cạnh
→ Số nút (nodes): 3,817 nút


In [52]:
clean_to_link = {}

def clean_title(name):
    name = name.replace("_", " ")
    name = re.sub(r'\b\w', lambda m: m.group().upper(), name)
    name = re.sub(r'\s+', ' ', name).strip(' ,.()')
    return name

for _, row in nodes.iterrows():
    link = row['link']
    if isinstance(link, str) and link.startswith("/wiki/"):
        clean_name = clean_title(link[6:])
        clean_to_link[clean_name] = link

print(f"→ Mapping name → link: {len(clean_to_link):,} nút")

→ Mapping name → link: 2,278 nút


In [53]:
person_clean_names = []

for _, row in nodes[nodes['type'] == 'PERSON'].iterrows():
    link = row['link']
    if isinstance(link, str) and link.startswith("/wiki/"):
        clean_name = clean_title(link[6:])
        person_clean_names.append(clean_name)

print(f"→ Số nhà khoa học PERSON: {len(person_clean_names):,}")

→ Số nhà khoa học PERSON: 1,957


In [54]:
person_edges = edges[
    edges['src'].isin(person_clean_names) |
    edges['des'].isin(person_clean_names)
].copy()

print(f"→ Số cạnh liên quan tới người: {len(person_edges):,}")

→ Số cạnh liên quan tới người: 9,310


In [55]:
G = nx.from_pandas_edgelist(person_edges, source='src', target='des', create_using=nx.DiGraph())

# Thêm tên hiển thị đẹp
display_name_map = {}
for clean, link in clean_to_link.items():
    row = nodes[nodes['link'] == link]
    if len(row) > 0:
        display_name_map[clean] = row.iloc[0]['name']

nx.set_node_attributes(G, display_name_map, 'display_name')

print(f"→ Đồ thị: {G.number_of_nodes():,} nút, {G.number_of_edges():,} cạnh\n")


→ Đồ thị: 4,509 nút, 8,716 cạnh



In [57]:
print("1. THẾ GIỚI NHỎ (Small-world)")

G_person = G.subgraph([n for n in person_clean_names if n in G]).copy()
G_und = G_person.to_undirected()

if len(G_und.nodes) == 0:
    print(" Không có node PERSON trong graph!")
else:
    if nx.is_connected(G_und):
        dist = nx.average_shortest_path_length(G_und)
        print(f"   Average shortest path = {dist:.3f} → RẤT NHỎ (Small World)")
    else:
        LCC = max(nx.connected_components(G_und), key=len)
        dist = nx.average_shortest_path_length(G_und.subgraph(LCC))
        print(f"   Largest Connected Component: {len(LCC)} người")
        print(f"   Average shortest path = {dist:.3f} → Small World rõ rệt")


1. THẾ GIỚI NHỎ (Small-world)
   Largest Connected Component: 89 người
   Average shortest path = 6.648 → Small World rõ rệt


In [58]:
print("\n2. TOP 20 NHÀ KHOA HỌC QUAN TRỌNG NHẤT (PageRank)")

pr = nx.pagerank(G, alpha=0.85)
person_pr = {k: v for k, v in pr.items() if k in person_clean_names}

top20 = sorted(person_pr.items(), key=lambda x: x[1], reverse=True)[:20]

print("   Hạng | Nhà khoa học                          | PageRank")
print("   ─────────────────────────────────────────────────────────────")

for i, (node, score) in enumerate(top20, 1):
    name = G.nodes[node].get('display_name', node)
    print(f"   {i:2d}.   {name:<38} {score:.6f}")


2. TOP 20 NHÀ KHOA HỌC QUAN TRỌNG NHẤT (PageRank)
   Hạng | Nhà khoa học                          | PageRank
   ─────────────────────────────────────────────────────────────
    1.   Adolf von Baeyer                       0.000593
    2.   Enrico Fermi                           0.000582
    3.   Emil Fischer                           0.000456
    4.   George Akerlof                         0.000411
    5.   Max von Laue                           0.000380
    6.   Den Helder                             0.000370
    7.   Martinus J. G. Veltman                 0.000370
    8.   Robert B. Wilson                       0.000363
    9.   Zamora                                 0.000355
   10.   Elias James Corey                      0.000355
   11.   Niels Kaj Jerne                        0.000355
   12.   Manne Siegbahn                         0.000355
   13.   Hermann Staudinger                     0.000355
   14.   Joseph John Thomson                    0.000336
   15.   Robert Solow      

In [59]:
print("\n3. PHÁT HIỆN CỘNG ĐỒNG (Louvain)")

G_und = G_person.to_undirected()

try:
    import community as community_louvain
    partition = community_louvain.best_partition(G_und)
    modularity = community_louvain.modularity(partition, G_und)
    print(f"   Modularity = {modularity:.3f}")
except:
    print("   Dùng greedy_modularity_communities (fallback)")
    coms = list(nx.community.greedy_modularity_communities(G_und))
    partition = {}
    for cid, com in enumerate(coms):
        for n in com:
            partition[n] = cid

sizes = Counter(partition.values())
print(f"   Tổng số cộng đồng: {len(sizes)}\n")

for cid, size in sizes.most_common(10):
    members = [n for n, c in partition.items() if c == cid][:8]
    names = [G.nodes[n].get('display_name', n) for n in members]
    print(f"   Cộng đồng {cid:2d}: {size:3d} người → {', '.join(names)} ...")


3. PHÁT HIỆN CỘNG ĐỒNG (Louvain)
   Dùng greedy_modularity_communities (fallback)
   Tổng số cộng đồng: 547

   Cộng đồng  0:  20 người → Gary Becker, Elinor Ostrom, Robert Fogel, Simon Kuznets, John Hicks, Friedrich Hayek, Alan Greenspan, Myron Scholes ...
   Cộng đồng  1:  18 người → Wassily Leontief, Peter A. Diamond, Vernon L. Smith, William Nordhaus, Janet Yellen, Richard Stone, Edmund Phelps, John Kenneth Galbraith ...
   Cộng đồng  2:  17 người → Francis William Aston, Frederick Soddy, William Shockley, Ernest Rutherford, Patrick Blackett, William Lawrence Bragg, Joseph John Thomson, Owen Willans Richardson ...
   Cộng đồng  3:  16 người → Linus Pauling, Martin Karplus, Felix Bloch, Thomas A. Steitz, Fritz Zwicky, David J. Thouless, Paul Dirac, Arnold Sommerfeld ...
   Cộng đồng  4:  15 người → Esther Duflo, Roger Myerson, John Harsanyi, Rabindranath Tagore, Joshua Angrist, Mahbub ul Haq, Barack Obama, Michael Spence ...
   Cộng đồng  5:  13 người → Otto Wallach, August Wilhelm